In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.models import Sequential, Model

from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

import os
from PIL import Image, ImageFile

import time


import matplotlib.pyplot as plt
import sklearn as sk

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
rom google.colab import drive
import glob

drive.mount('/content/gdrive')

path = "/content/gdrive/My Drive/Colourize Autoencoder/Flower Training Set"

datagen = ImageDataGenerator(rescale=1./255)
train_data = datagen.flow_from_directory(path, target_size=(256, 256),batch_size=99,class_mode='input')


In [ ]:
l = []
ab = []
x = 0
for image in train_data[0][0]:
    try:
        plt.imshow(image)
        x+=1
        lab = rgb2lab(image)
        l.append(lab[:,:,0])
        ab.append(lab[:,:,1:]/128)
    except:
        print("Error in converting rgb image to lab")

print("X: " + str(x))
print("SHAPES")

#converting lists l and ab to np.arrays
l = np.array(l)
ab = np.array(ab)

print("L  Shape: " + str(l.shape))
print("AB Shape: " + str(ab.shape))

#reshaping numpy array np_l
l = l.reshape(l.shape + (1,))
print("*AFTER FIXING SHAPE*")
print("L  Shape: " + str(l.shape) + "\n")

In [ ]:
#Encoder
en_input = Input(shape=(256, 256, 1,))
en_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(en_input)
en_output = Conv2D(128, (3,3), activation='relu', padding='same')(en_output)
en_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(en_output)
en_output = Conv2D(256, (3,3), activation='relu', padding='same')(en_output)
en_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(en_output)
en_output = Conv2D(512, (3,3), activation='relu', padding='same')(en_output)
en_output = Conv2D(512, (3,3), activation='relu', padding='same')(en_output)
en_output = Conv2D(256, (3,3), activation='relu', padding='same')(en_output)


In [ ]:
#Decoder
dec_output = Conv2D(128, (3,3), activation='relu', padding='same')(en_output)
dec_output = UpSampling2D((2, 2))(dec_output)
dec_output = Conv2D(64, (3,3), activation='relu', padding='same')(dec_output)
dec_output = UpSampling2D((2, 2))(dec_output)
dec_output = Conv2D(32, (3,3), activation='relu', padding='same')(dec_output)
dec_output = Conv2D(16, (3,3), activation='relu', padding='same')(dec_output)
dec_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(dec_output)
dec_output = UpSampling2D((2, 2))(dec_output)
model = Model(inputs=en_input, outputs=dec_output)

In [ ]:
#Training the data using MSE
start = time.time()
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.fit(l,ab,validation_split=0.15, epochs=100) #1000
end = time.time()
print(end-start)

In [ ]:
#post processing (do not have test data so just going to use train data for now)
test_path = '/content/gdrive/My Drive/Colourize Autoencoder/Flower Testing Dataset/Test (Colour)/'
test_path = '/content/gdrive/My Drive/Colourize Autoencoder/Flower Testing Dataset/Test (B-W)/'

test_data = os.listdir(test_path)
color_me = []
for img_name in test_data:
    print(img_name)
    image = img_to_array(load_img(test_path + img_name))
    image = resize(image ,(256,256))
    color_me.append(image)

color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
#print(color_me.shape)

output = model.predict(color_me)
print(output.shape)
output *= 128
print(output)

In [ ]:
torch.save(model, format("/content/gdrive/My Drive/TESTSAVE"))

In [ ]:
#Reconstructing the Lab image - Output colorizations
for i in range(len(output)):
    result = np.zeros((256, 256, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    #imsave("result"+str(i)+".png", lab2rgb(result))
    plt.imshow(lab2rgb(result))

In [ ]:
i=3
result[:,:,0] = color_me[i][:,:,0]
result[:,:,1:] = output[i]
#imsave("result"+str(i)+".png", lab2rgb(result))
plt.imshow(lab2rgb(result))




# result = np.zeros((256, 256, 3))
# result[:,:,1:] = output[i]
# plt.imshow(result)
# result[:,:,0] = color_me[i][:,:,0]
#imsave("result"+str(i)+".png", lab2rgb(result))
#plt.imshow(lab2rgb(result))